# Загрузка Pandas и очистка данных

In [17]:
import pandas as pd
import re
df = pd.read_csv('main_task.csv')

In [28]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
#'Reviews', 'URL_TA', 'ID_TA', 'Rating', 'Restaurant_id'

X = df.drop(['Reviews', 'URL_TA', 'ID_TA', 'Rating'], axis=1)

#Первый признак: Индекс города

list_city = list(df['City'].unique())
dict_city = {}
for each in list_city:
    dict_city[each] = len(dict_city)
X['City'] = X['City'].apply(lambda x: dict_city[x])

#Второй признак: Уровень цен
df['Price Range'] = df['Price Range'].fillna(0)
price_dict = {'$$ - $$$': 5, '$$$$': 15, '$': 3}
X['Price Range'] = df['Price Range'].apply(lambda x: 3 if x == 0 else price_dict[x])

#Третий признак: Количество кухонь
df['Cuisine Style'].fillna("", inplace=True)
X['Cuisine Style_'] = df['Cuisine Style'].apply(lambda x: len(re.findall('\w+\s*\w+\s*\w+', str(x))))

#Четвертый признак: Встречаются ли в отзывах слова Good|good|excellent|Excellent|awesome|Awesome|Best|best|Nice|nice
                                                 
X['is_good'] = df['Reviews'].apply(lambda x: 0 if pd.isna(re.search("Good|good|excellent|Excellent|awesome|Awesome|Best|best|Nice|nice", x)) else 1)

def review_time(row):
    match = re.findall('\d\d/\d\d/\d\d\d\d', row['Reviews'])
    if len(match) == 0:
        return [pd.to_datetime('10/25/2017', format='%m/%d/%Y'), pd.to_datetime('10/25/2017', format='%m/%d/%Y')]
    elif len(match) == 1:
        return [pd.to_datetime(match[0], format='%m/%d/%Y'), pd.to_datetime('10/25/2017', format='%m/%d/%Y')]
    return [pd.to_datetime(match[0], format='%m/%d/%Y'), pd.to_datetime(match[1], format='%m/%d/%Y')]
df2 = df.apply(review_time, axis=1, result_type='expand')
fake = pd.to_datetime('10/25/2017',  format='%m/%d/%Y')
X['delta'] = df2.apply(lambda x: (x[0]-x[1]).days if not x[0] == fake or not x[1] == fake else 100000, axis=1)

In [29]:
X = X.drop(['Cuisine Style'], axis=1)
X['Restaurant_id'] = X['Restaurant_id'].apply(lambda x: int(str(x)[3:]))
X = X.fillna(0)

In [30]:
X.head()


,Restaurant_id,City,Ranking,Price Range,Number of Reviews,Cuisine Style_,is_good,delta
0,5569,0,5570.0,5,194.0,3,1,41
1,1535,1,1537.0,3,10.0,0,0,382
2,352,2,353.0,15,688.0,7,0,2
3,3456,3,3458.0,3,3.0,0,0,100000
4,615,4,621.0,5,84.0,3,1,272


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [31]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
y = df['Rating']

In [32]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Создаём, обучаем и тестируем модель

In [33]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Создаём модель
regr = RandomForestRegressor(n_estimators=100, random_state=42)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21335549999999998
